In [2]:
%classpath add jar ../../konduit.jar

In [3]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.pipeline.api.data.Data;
import ai.konduit.serving.pipeline.api.data.Image;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;
import ai.konduit.serving.pipeline.impl.format.JavaImageFactory;
import ai.konduit.serving.pipeline.registry.ImageFactoryRegistry;
import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

public class OCREndPoint implements Endpoint {

    private PipelineExecutor pipelineExecutor;

    public OCREndPoint(PipelineExecutor pipelineExecutor) { this.pipelineExecutor = pipelineExecutor; ImageFactoryRegistry.addFactory(new JavaImageFactory()); }

    public HttpMethod type() { return HttpMethod.POST; }

    public String path() { return "/infer"; }

    public List<String> consumes() { return Arrays.asList("application/octet-stream","multipart/form-data"); }

    public List<String> produces() { return Arrays.asList("application/json"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> {
            handler.vertx().executeBlocking(taskHandler -> {
                Data image = Data.empty();
                
                try {
                    image.put("image",Image.create(ImageIO.read(new File(handler.fileUploads().iterator().next().uploadedFileName()))));
                } catch (IOException e) {
                    e.printStackTrace();
                }

                Data exec = pipelineExecutor.exec(image);
                
                handler.response().end(exec.toJson());
                taskHandler.complete();
            },resultHandler -> {
                if(resultHandler.failed()) {
                    if(resultHandler.cause() != null)
                        if(handler.vertx().exceptionHandler() != null)
                            handler.vertx().exceptionHandler().handle(resultHandler.cause());
                        else {
                            resultHandler.cause().printStackTrace();
                        }
                    else {
                        System.err.println("Failed to process classification endpoint async task. Unknown cause.");
                    }
                }
            });

        };
    }
}

ai.konduit.OCREndPoint

In [4]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.endpoint.HttpEndpoints;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import java.util.Arrays;
import java.util.List;

public class OCREndPoints implements HttpEndpoints {

    @Override
    public List<Endpoint> endpoints(Pipeline pipeline, PipelineExecutor pipelineExecutor) {
        return Arrays.asList(new OCREndPoint(pipelineExecutor));
    }
}

ai.konduit.OCREndPoints

In [5]:
import java.net.URLClassLoader;
import java.net.URL;

URL[] urls = ((URLClassLoader) Class.forName("ai.konduit.serving.vertx.config.InferenceConfiguration").getClassLoader()).getURLs();

for(URL url : urls) {
    System.out.println(new java.io.File(url.toURI()));
}

/tmp/beaker3211561742937850753/outDir
/root/konduit/konduit.jar


null

In [12]:
%%bash
java -cp /tmp/beaker3211561742937850753/outDir:../../konduit.jar ai.konduit.serving.cli.launcher.KonduitServingLauncher serve -id onnx-mnist -c config.json -rwm -b

Starting konduit server...
Using classpath: /tmp/beaker3211561742937850753/outDir:../../konduit.jar
INFO: Running command /opt/conda/jre/bin/java -Dkonduit.logs.file.path=/root/.konduit-serving/command_logs/onnx-mnist.log -Dlogback.configurationFile=/tmp/logback-run_command_376b411a13ed4632.xml ai.konduit.serving.cli.launcher.KonduitServingLauncher run --instances 1 -s inference -c config.json -Dserving.id=onnx-mnist
For server status, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher list'
For logs, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher logs onnx-mnist'



In [13]:
%%bash
konduit list


Listing konduit servers...

 #   | ID                             | TYPE       | URL                  | PID     | STATUS     
 1   | server                         | inference  | 0.0.0.0:8082         | 75      | started    
 2   | onnx-iris                      | inference  | localhost:33641      | 258     | started    
 3   | onnx-mnist                     | inference  | localhost:40733      | 543     | started    




In [16]:
%%bash
konduit logs onnx-mnist --lines 100

03:25:22.012 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Processing configuration: /root/konduit/demos/2-pytorch-onnx-mnist/config.json
03:25:22.020 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Replaced standard System.out and System.err PrintStreams with SLF4JPrintStreams
03:25:22.021 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Redirected System.out and System.err to SLF4J for this context
03:25:22.021 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Starting konduit server with an id of 'onnx-mnist'
03:25:22.294 [vert.x-worker-thread-0] INFO  a.k.s.p.registry.PipelineRegistry - Loaded 27 PipelineStepRunnerFactory instances
03:25:22.498 [vert.x-worker-thread-0] INFO  a.k.s.v.verticle.InferenceVerticle - 

####################################################################
#                                                                  #
#    |  /   _ \   \ |  _ \  |  | _ _| __ __|    |  /     |  /      #
#    . <   (   | .  |  |  | |  |   |     |      . <      . <   

In [18]:
%%bash
curl -s -H "Content-Type: multipart/form-data" -X POST -F 'image=@test-image.jpg' http://localhost:$(konduit inspect onnx-mnist -q {port})/infer

{
  "Plus214_Output_0" : {
    "@NDArrayType" : "FLOAT",
    "@NDArrayShape" : [ 1, 10 ],
    "@NDArrayDataBase64" : "DXDlv7BvGsH7pzdBlsSkQML5/cAqHBxBB2ULPyfT9cBpwRNBsNLNwA=="
  }
}

